<a href="https://colab.research.google.com/github/kirbu123/FilmGeneration/blob/develop/FilmGenerationAttemp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import usefull libs

In [1]:
import torch

In [2]:
import pandas as pd
from google.colab import drive
import os
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import numpy as np
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedKFold
from sklearn.multiclass import OneVsRestClassifier

## Load dataset